In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
'''
!pip install python-dotenv
!pip install neo4j
!pip install pandas
!pip install langchain
!pip install transformers
!pip install langchain.chains
'''

# Note: will do a requirements.txt file later

'\n!pip install python-dotenv\n!pip install neo4j\n!pip install pandas\n!pip install langchain\n!pip install transformers\n!pip install langchain.chains\n'

In [19]:
from dotenv import load_dotenv
import os
from neo4j import GraphDatabase
import pandas as pd
from xlsx_to_csv import *
import warnings
warnings.filterwarnings("ignore")

load_dotenv()

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

URI = NEO4J_URI
AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [20]:
from kg import *

create_kg()

Knowledge Graph created with 33 nodes and 96 relationships.


In [21]:
def query_knowledge_graph(query):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (n:Person)-[r]->(m:Person)
            WHERE n.name CONTAINS $query OR m.name CONTAINS $query
            RETURN n.name, r, m.name
            """,
            query=query
        )
        return result.data()


In [22]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

graph.refresh_schema()

print(graph.schema)

Node properties:
Person {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:TEAMMATE]->(:Person)
(:Person)-[:FRIEND]->(:Person)
(:Person)-[:CLASSMATE]->(:Person)
(:Person)-[:FAMILY]->(:Person)
(:Person)-[:IMMEDIATE_FAMILY]->(:Person)
(:Person)-[:LOVER]->(:Person)
(:Person)-[:OPPONENT]->(:Person)
(:Person)-[:ENEMY]->(:Person)
(:Person)-[:TEACHER]->(:Person)


In [26]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

llm = llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({'query': 'List all of Harry\'s friends'})
response



> Entering new GraphCypherQAChain chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
from langchain.chains import RetrievalQA
from langchain_community.llms import OpenAI

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

retrieval_qa = RetrievalQA(llm=OpenAI(OPENAI_API_KEY), knowledge_graph=query_knowledge_graph)


TypeError: BaseOpenAI.__new__() takes 1 positional argument but 2 were given